In [67]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools, ConfluentTableDescriptor
from pyflink.table import TableEnvironment, DataTypes, Schema
from pyflink.table.expressions import *
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.window import Tumble, Slide, Session
from pyflink.table.udf import udf

In [68]:
from dotenv import load_dotenv
load_dotenv()
# load_dotenv("/Users/dwoodbridge/Class/2025_MSDS682/Examples/msdes682_data_stream_processing_2025/Day11/.env")

True

| +I |                              2 |                           65.1 | 2025-11-21 05:36:41.000 | 2025-11-21 05:36:42.000 |
| +I |                              3 |                           65.2 | 2025-11-21 05:36:42.000 | 2025-11-21 05:36:43.000 |
| +I |                              2 |                           65.1 |                    1 | 2025-11-21 05:36:39.000 | 2025-11-21 05:36:42.000 |
| +I |                              1 |                           65.2 | 2025-11-21 05:36:43.000 | 2025-11-21 05:36:44.000 |
| +I |                              4 |                           65.2 | 2025-11-21 05:36:44.000 | 2025-11-21 05:36:45.000 |
| +I |                              5 |                           65.2 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:46.000 |
| +I |                              4 |                           65.2 |                    1 | 2025-11-21 05:36:42.000 | 2025-11-21 05:36:45.000 |
| +I |                              1 |                           65.2 |       

In [69]:
import os
print(os.getenv("CLOUD_REGION"))

us-west1


In [70]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

In [71]:
env.use_catalog("msds682")
env.use_database("msds682")
env.execute_sql("""DROP TABLE IF EXISTS `system.usage.new`""")

In [72]:
env.execute_sql("""
    CREATE TABLE IF NOT EXISTS `system.usage.new` (
        id STRING,
        cpu_usage FLOAT,
        cpu_stats ARRAY<FLOAT>,
        memory_usage FLOAT,
        `timestamp` TIMESTAMP_LTZ(3),
        WATERMARK FOR `timestamp` AS `timestamp` - INTERVAL '5' SECOND
    ) WITH (
        'value.format'='avro-registry',
        'connector' = 'confluent'
    )
""")

In [73]:
table = env.from_path("`system.usage.new`")

In [ ]:
# from pyflink.table.expressions import col, lit,interval

windowed_table = table.window(Tumble.over(lit(3).second)
            .on(col("timestamp"))
            .alias("w")           
)
result = windowed_table.group_by(
    col("w"), 
    col("id")
).select(
    col("id"),
    col("memory_usage").min.alias("min_memory_usage"),
    col("memory_usage").max.alias("max_memory_usage"), # Access 'max' as a property
    col("memory_usage").count.alias("record_ct"),
    col("w").start.alias("window_start"),              
    col("w").end.alias("window_end")                   
)

| +I |                              5 |                           65.2 |                    1 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:48.000 |
| +I |                              4 |                           65.2 |                    1 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:48.000 |
| +I |                              4 |                           65.0 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              5 |                           65.2 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              2 |                           65.1 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              3 |                           65.4 |                    1 | 2025-11-21 05:36:51.000 | 2025-11-21 05:36:54.000 |
| +I |                              4 |                           65.2 |                           65.2 |       

In [75]:
ConfluentTools.print_changelog(result)

| +I |                              4 |                           65.5 | 2025-11-21 05:37:13.000 | 2025-11-21 05:37:14.000 |
| +I |                              3 |                           65.5 | 2025-11-21 05:37:14.000 | 2025-11-21 05:37:15.000 |
| +I |                              4 |                           65.6 | 2025-11-21 05:37:15.000 | 2025-11-21 05:37:16.000 |
| +I |                              4 |                           65.5 |                    1 | 2025-11-21 05:37:13.000 | 2025-11-21 05:37:14.000 |
| +I |                              3 |                           65.5 |                    1 | 2025-11-21 05:37:14.000 | 2025-11-21 05:37:15.000 |
| +I |                              4 |                           65.6 |                    1 | 2025-11-21 05:37:15.000 | 2025-11-21 05:37:16.000 |
| +I |                              4 |                           65.5 |                    1 | 2025-11-21 05:37:12.000 | 2025-11-21 05:37:15.000 |
| +I |                           

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

| +I |                              2 |                           65.7 |                           65.7 |                    1 | 2025-11-21 05:37:54.000 | 2025-11-21 05:37:57.000 |


In [13]:
# from pyflink.table.expressions import col, lit
# from pyflink.table.window import Tumble

# # ... (rest of your table definition code)

# windowed_table = table.window(Tumble.over(lit(10).seconds)
#           .on(col("$rowtime"))
#           .alias("w")           
# )
# result_table = windowed_table.group_by( # Renamed variable for clarity
#     col("w"), 
#     col("id")
# ).select(
#     col("id"), 
#     col("memory_usage").max.alias("max_memory_usage"), # Access 'max' as a property
#     col("w").start.alias("window_start"),              
#     col("w").end.alias("window_end")                   
# )

# # Execute the table operation and get a TableResult object
# table_result = result_table.execute()

# # Now collect the results from the TableResult
# max_records = 10
# records_collected = 0

# print("Starting data collection (streaming mode)...")

# # Use a 'with' statement to ensure the iterator and job are properly closed/cancelled
# with table_result.collect() as results:
#     for row in results:
#         print(row)
#         records_collected += 1
#         if records_collected >= max_records:
#             print(f"Collected {max_records} records. Terminating manually.")
#             break # Breaking the loop will implicitly call close() on the iterator and cancel the job
        
# print("Job execution finished.")


In [14]:
# from pyflink.table.expressions import col, lit
# from pyflink.table.window import Tumble

# # Keep the runtime mode as STREAMING (default) or don't set it to batch.
# # The job is fundamentally a streaming job because the source is unbounded.

# windowed_table = table.window(Tumble.over(lit(1).second)
#           .on(col("$rowtime"))
#           .alias("w")           
# )
# result = windowed_table.group_by(
#     col("w"), 
#     col("id")
# ).select(
#     col("id"), 
#     col("w").start.alias("window_start"),              
#     col("w").end.alias("window_end")                   
# )

# # Use result.collect() and manually stop after reaching the limit
# max_records = 10  # Set your desired limit
# records_collected = 0

# print("Starting data collection (streaming mode)...")

# try:
#     for row in result.collect():
#         print(row)
#         records_collected += 1
#         if records_collected >= max_records:
#             print(f"Collected {max_records} records. Terminating manually.")
#             # In a notebook, the loop breaking usually stops the collection internally.
#             break
# except Exception as e:
#     # Handle potential exceptions gracefully if the job is forcibly shut down
#     print(f"Collection stopped due to exception: {e}")

# print("Job execution finished.")


In [18]:
from pyflink.table import StreamTableEnvironment, DataTypes
from pyflink.table.expressions import col, lit
from pyflink.table.window import Tumble
from dotenv import load_dotenv
import os

load_dotenv()
# Assuming t_env (StreamTableEnvironment) is already created
# t_env = StreamTableEnvironment.create(stream_execution_environment=env)

topic_name = "system.usage.flink"
# ... other Kafka configs (bootstrap servers, security protocol, etc.) ...

source_ddl = f"""
CREATE TABLE system_usage_source (
    id STRING,
    cpu_usage FLOAT,
    memory_usage FLOAT,
    -- Define the raw timestamp field as a string (matching your Avro schema)
    timestamp_str STRING,
    -- Define a computed column to convert the string to a proper TIMESTAMP_LTZ(3)
    -- This uses the standard SQL CAST function
    event_time AS CAST(timestamp_str AS TIMESTAMP_LTZ(3)),
    -- Define the watermark strategy for the event_time column
    -- This example assumes data might be up to 5 seconds out of order
    WATERMARK FOR event_time AS event_time - INTERVAL '5' SECOND
) WITH (
    'connector' = 'kafka',
    'topic' = '{topic_name}',
    'properties.bootstrap.servers' = {os.getenv("BOOTSTRAP_SERVERS")},
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'avro-confluent',
    'avro-confluent.schema-registry.url' = {os.getenv("SCHEMA_REGISTRY_URL")}
)
"""

env.execute_sql(source_ddl)

# Access the table with the defined 'event_time' attribute
table = t_env.from_path("system_usage_source")

# Now you can use the 'event_time' column for windowing
windowed_table = table.window(
    Tumble.over(lit(1).second)
          .on(col("event_time")) # Use the new time attribute column
          .alias("w")           
)

result_table = windowed_table.group_by(
    col("w"), 
    col("id")
).select(
    col("id"),
    col("w").start.alias("window_start"),              
    col("w").end.alias("window_end")                   
)

# Use the collect mechanism to consume the finite set of records produced by your producer
table_result = result_table.execute()
max_records = 10
records_collected = 0

print("Starting data collection (streaming mode with watermarks)...")

with table_result.collect() as results:
    for row in results:
        print(row)
        records_collected += 1
        if records_collected >= max_records:
            print(f"Collected {max_records} records. Terminating manually.")
            break
        
print("Job execution finished.")


Py4JJavaError: An error occurred while calling o44.executeSql.
: io.confluent.flink.plugin.ConfluentFlinkException: Statement 'table-api-2025-11-20-115433-7616b2eb-8818-46c2-82fd-00926e7a9ee7-dry' failed.
Reason: SQL parse failed. Encountered "None" at line 17, column 38.
Was expecting one of:
    <BINARY_STRING_LITERAL> ...
    <QUOTED_STRING> ...
    <PREFIXED_STRING_LITERAL> ...
    <UNICODE_STRING_LITERAL> ...
    <BIG_QUERY_DOUBLE_QUOTED_STRING> ...
    <BIG_QUERY_QUOTED_STRING> ...
    
	at io.confluent.flink.plugin.internal.DefaultPluginContext.lambda$statementStatusCondition$10(DefaultPluginContext.java:529)
	at io.confluent.flink.plugin.internal.DefaultPluginContext.submitStatement(DefaultPluginContext.java:417)
	at io.confluent.flink.plugin.internal.DefaultPluginContext.queryDryRun(DefaultPluginContext.java:212)
	at io.confluent.flink.plugin.internal.ConfluentParser.parse(ConfluentParser.java:47)
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.executeSql(TableEnv

In [ ]:
"""
How to View Intermediate Results from Streaming Flink Queries
Solutions for windowed aggregations that run forever
"""
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools
from pyflink.table import TableEnvironment
from pyflink.table.expressions import col, lit
from pyflink.table.window import Tumble

load_dotenv()
settings = ConfluentSettings.from_global_variables()
env = TableEnvironment.create(settings)

print("=" * 80)
print("VIEWING INTERMEDIATE RESULTS FROM STREAMING QUERIES")
print("=" * 80)
print()

print("""
╔════════════════════════════════════════════════════════════════════════════╗
║  THE PROBLEM: Streaming queries with windows run FOREVER                  ║
║  They continuously process new windows as data arrives                     ║
╚════════════════════════════════════════════════════════════════════════════╝
""")

print()

# ============================================================================
# SOLUTION 1: Use ConfluentTools.print_changelog_limit() (BEST for Testing)
# ============================================================================
print("SOLUTION 1: Use ConfluentTools.print_changelog_limit() [RECOMMENDED]")
print("=" * 80)
print()

solution1 = '''
from pyflink.table.confluent import ConfluentTools
from pyflink.table.expressions import col, lit
from pyflink.table.window import Tumble

env.use_catalog("msds682")
env.use_database("msds682")
table = env.from_path("`system.usage`")

# Your windowed aggregation
result = (
    table
    .window(Tumble.over(lit(10).seconds).on(col("$rowtime")).alias("w"))
    .group_by(col("id"), col("w"))
    .select(
        col("id"),
        col("w").start.alias("window_start"),
        col("w").end.alias("window_end"),
        col("memory_usage").min.alias("min_mem")
    )
)

# ✓ This will print first 20 windows and then STOP
ConfluentTools.print_changelog_limit(result, 20)

print("✓ Done! Printed 20 windows and stopped")
'''

print(solution1)
print()
print("Why this works:")
print("• ConfluentTools.print_changelog_limit() bounds the query")
print("• It stops after collecting N results")
print("• Perfect for testing and debugging")
print()

# ============================================================================
# SOLUTION 2: Add LIMIT to bound the query
# ============================================================================
print("SOLUTION 2: Add .limit() to Your Query")
print("=" * 80)
print()

solution2 = '''
result = (
    table
    .window(Tumble.over(lit(10).seconds).on(col("$rowtime")).alias("w"))
    .group_by(col("id"), col("w"))
    .select(
        col("id"),
        col("w").start.alias("window_start"),
        col("w").end.alias("window_end"),
        col("memory_usage").min.alias("min_mem")
    )
    .limit(10)  # ✓ Add this to stop after 10 windows
)

# Now you can use regular execute()
result.execute().print()
# Or
ConfluentTools.print_changelog_limit(result, 10)
'''

print(solution2)
print()
print("Why this works:")
print("• .limit(N) makes the query bounded")
print("• Query stops after N results")
print("• Returns control to your notebook")
print()

# ============================================================================
# SOLUTION 3: Write to a Kafka Topic (for Production)
# ============================================================================
print("SOLUTION 3: Write Results to Kafka Topic [PRODUCTION]")
print("=" * 80)
print()

solution3 = '''
# Step 1: Create output table (Kafka topic)
create_output = """
    CREATE TABLE IF NOT EXISTS window_results (
        id STRING,
        window_start TIMESTAMP(3),
        window_end TIMESTAMP(3),
        min_mem DOUBLE,
        PRIMARY KEY (id, window_start) NOT ENFORCED
    ) WITH (
        'changelog.mode' = 'upsert',
        'kafka.retention.time' = '7 d'
    )
"""
env.execute_sql(create_output)

In [6]:
result = (
    table
    .window(Tumble.over(lit(10).seconds).on(col("rowtime")).alias("w"))
    .group_by(col("id"), col("w"))
    .select(
        col("id"),
        col("w").start.alias("window_start"),
        col("w").end.alias("window_end"),
        min(col("memory_usage")).alias("min_mem")
    )
)
ConfluentTools.print_changelog_limit(result, 20)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 